<a href="https://colab.research.google.com/github/aslan-ng/CheeseMate/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies

!pip -q install --upgrade "smolagents[transformers]" gradio

In [ ]:
# Make imports

import re
import gradio as gr
from smolagents import TransformersModel

In [ ]:
# Create model (Qwen)

PARAMETERS_COUNT = 0.5  # Billions
MODEL_ID = f"Qwen/Qwen2.5-{PARAMETERS_COUNT}B-Instruct"

model = TransformersModel(
    model_id=MODEL_ID,
    device_map="auto",
    torch_dtype="auto",
    max_new_tokens=512,
    temperature=0.2,
)

In [ ]:
# Propmts and examples

SYSTEM = """
You are CheeseMatch, a friendly assistant that ONLY recommends cheeses.
- Stay strictly in the cheese domain; if asked anything else, politely redirect.
- Be concise and conversational (1–3 sentences).
- Give 1–3 cheese suggestions max, with a brief why (taste/texture/use/diet notes).
- Prefer widely available cheeses unless the user requests niche options.
- If constraints are unclear, ask ONE short clarification question.
- Never output JSON, tables, or code.
"""

FEWSHOT = """
User: Where is the capital of France?
Assistant: I can only help you select good cheese! Do you want me to suggest a cheese?

User: I'm lactose intolerant and I don't like salty cheese. What do you recommend?
Assistant: Consider naturally low-lactose, milder cheeses like Swiss-style options; they tend to be easier on lactose and not overly salty.

User: Low fat, high protein cheeses
Assistant: Cottage cheese is a solid choice—high in protein and available in low-fat varieties.
"""

In [ ]:
# Main body of program that combines them all

def _to_text(reply) -> str:
    """
    Function to convert smolagents ChatMessage (or str) to plain text.
    """
    if isinstance(reply, str):
        return reply
    # Try common ChatMessage shapes
    try:
        content = reply.content  # may be str or list of parts
        if isinstance(content, str):
            return content
        if isinstance(content, list):
            # look for a text-like part
            for part in content:
                if isinstance(part, dict):
                    if part.get("type") in {"text", "output_text"} and "text" in part:
                        return part["text"]
                elif hasattr(part, "text"):
                    return part.text
    except Exception:
        pass
    # Fallback
    return str(reply)


class CheeseChat:
    """
    CheeseMatch assistant.
    """
    def __init__(self):
        self.awaiting_confirm = False

    def respond(self, user_text: str) -> str:
        t = user_text.strip()
        tl = t.lower()

        # If we previously redirected and the user confirms
        if self.awaiting_confirm and tl in {"yes", "yes!", "ok", "okay", "sure", "yep"}:
            self.awaiting_confirm = False
            prompt = FEWSHOT + "\n\nUser: Please suggest a good cheese.\nAssistant:"
            reply = model.generate(
                messages=[
                    {"role": "system", "content": [{"type": "text", "text": SYSTEM}]},
                    {"role": "user",   "content": [{"type": "text", "text": prompt}]},
                ],
            )
            return _to_text(reply).strip()

        # Default: cheese-only assistant behavior (FEWSHOT carries the redirect example)
        prompt = FEWSHOT + f"\n\nUser: {t}\nAssistant:"
        reply = model.generate(
            messages=[
                {"role": "system", "content": [{"type": "text", "text": SYSTEM}]},
                {"role": "user",   "content": [{"type": "text", "text": prompt}]},
            ],
        )
        text = _to_text(reply).strip()

        # If the model chose to redirect (e.g., off-topic question),
        # remember to expect a confirmation on the next turn.
        if "do you want me to suggest a cheese" in text.lower():
            self.awaiting_confirm = True

        return text

In [ ]:
# Create the bot

bot = CheeseChat()

In [ ]:
# Some examples without GUI

#message = "I'm lactose intolerant and I don't like salty cheese. what do you recommend?"
#print(message)
#print(bot.respond(message))

#message = "Where is the capital of France?"
#print(message)
#print(bot.respond(message))

In [ ]:
# Create GUI

with gr.Blocks(theme="soft") as demo:
    gr.Markdown("## 🧀 CheeseMatch\nI only help you select good cheese. Ask away!")

    chat = gr.Chatbot(height=420, type="messages")  # type='messages' keeps roles tidy
    txt = gr.Textbox(placeholder="Type your message about cheese…", autofocus=True)
    clear = gr.Button("Clear")

    def user_submit(user_message, history):
        # history is a list of dicts: [{"role":"user"/"assistant","content":...}, ...]
        history = history or []
        history.append({"role": "user", "content": user_message})
        bot_reply = bot.respond(user_message)
        history.append({"role": "assistant", "content": bot_reply})
        return gr.update(value=history), gr.update(value="")

    def clear_fn():
        # Reset bot state between conversations if you want
        global bot
        bot = CheeseChat()
        return [], ""

    txt.submit(user_submit, [txt, chat], [chat, txt])
    clear.click(clear_fn, [], [chat, txt])

demo.launch(share=True)